In [67]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os.path

In [17]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
print(url)

https://en.wikipedia.org/wiki/List_of_S%26P_500_companies


In [59]:
resp = requests.get(url)

In [60]:
if resp.status_code != 200:
    print('Server sent bad response. HTTP Status Code: {}'.format(resp.status_code))

In [61]:
soup = BeautifulSoup(resp.text, features='html.parser')

In [62]:
table = soup.find('table', {'id' : 'constituents'})
tbody = table.find('tbody')

data = {}
col_names = []

ths = tbody.find('tr').find_all('th')

for idx, th in enumerate(ths):
    if idx == 0 or idx == len(ths)-1:
        data[th.text[:-1]] = []
        col_names.append(th.text[:-1])
    else:
        data[th.text] = []
        col_names.append(th.text)
    
print(data)
print(col_names)

{'Symbol': [], 'Security': [], 'SEC filings': [], 'GICS Sector': [], 'GICS Sub-Industry': [], 'Headquarters Location': [], 'Date first added': [], 'CIK': [], 'Founded': []}
['Symbol', 'Security', 'SEC filings', 'GICS Sector', 'GICS Sub-Industry', 'Headquarters Location', 'Date first added', 'CIK', 'Founded']


In [63]:
trs = tbody.find_all('tr')[1:]

for tr in trs:
    tds = tr.find_all('td')
    #print(tds)
    for idx, td in enumerate(tds):
        #print(col_names[idx])
        #print(type(data[col_names[idx]]))
        if idx == 0 or idx == len(tds)-1:
            data.get(col_names[idx]).append(td.text[:-1])
        else:
            data.get(col_names[idx]).append(td.text)    
        
df = pd.DataFrame(data)

In [64]:
df.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,0000066740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,0000001800,1888
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,0001551152,2013 (1888)
3,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,0000815094,1981
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373,1989


In [65]:
df.drop(['SEC filings', 'CIK', 'Date first added'], axis=1, inplace=True)

In [66]:
df.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1902
1,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1888
2,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2013 (1888)
3,ABMD,Abiomed,Health Care,Health Care Equipment,"Danvers, Massachusetts",1981
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",1989


In [70]:
df.to_excel('~/Documents/S&P-500-list.xlsx')